# Fraction Differentiation - frac_diff_transf.py

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from fin_ml.transformers import frac_diff_transf

## Creating the DataFrame

In [2]:
lenght = int(1e3)
# creating series from a Normal distribution for the returns
df_ret1 = pd.Series(np.random.normal(0.0, 0.01,lenght))
df_ret2 = pd.Series(np.random.normal(0.0, 0.02,lenght))
# creating the price series
df_price1 = (1 + df_ret1).cumprod()
df_price2 = (1 + df_ret2).cumprod()
# concatenating the series into a DataFrame
X = pd.concat([df_price1, df_price2],axis = 1)
X = X.rename(columns={0:'X1', 1:'X2'})

## Fractional Differentiating

In [3]:
# Instantiating the FracDiff transformer
data_processor = frac_diff_transf.FracDiff(minimum = True)
# Transforming the Data
X_transf = data_processor.transform(X)
X_transf.head(10)

,X1_fd,X2_fd
12,0.372746,1.077526
13,0.386978,1.074479
14,0.400694,1.080775
15,0.404026,1.075664
16,0.412193,1.053125
17,0.419789,1.030055
18,0.415273,1.043604
19,0.403292,1.038935
20,0.404711,1.044504
21,0.416031,0.999612


#  Scikit-learn Pipeline for ColumnTransformer Using frac_diff_transf

## Importing Additional Libraries

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

## Creating and Running Pipeline

In [8]:
features = ['X1', 'X2']

# instantiating the transformer
transformer = frac_diff_transf.FracDiff(d = 1., 
                                        method = 'fww', 
                                        thres = 1e-2,
                                        minimum = True,
                                        min_dict = {'interv': [0., 1.], 
                                                    'step': 1e-1, 
                                                    'c_val_idx': int(1)})

# pipeline for the transformer
pipe_frac_diff = Pipeline([("frac_diff", transformer)])
                           
# column transformer using the pipeline transformer
pre_processor = ColumnTransformer([("frac_diff", pipe_frac_diff, features)], remainder ='passthrough')

# instantiating the pipeline of ColumnTransformer
pipe = Pipeline([("pp", pre_processor)])

# trasnforming the series
X_transf = pipe.fit_transform(X)
X_transf.head(10)

,frac_diff__X1_fd,frac_diff__X2_fd
12,0.372746,1.077526
13,0.386978,1.074479
14,0.400694,1.080775
15,0.404026,1.075664
16,0.412193,1.053125
17,0.419789,1.030055
18,0.415273,1.043604
19,0.403292,1.038935
20,0.404711,1.044504
21,0.416031,0.999612
